In [1]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
import plotly.graph_objects as go
# from better_profanity import profanity
import string


# added from here
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB, BernoulliNB


from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import nltk
import sklearn
import nltk.collocations 
from nltk import FreqDist, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string, re
import urllib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb
#nltk.download('punkt')
#nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')


from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import Ridge #import ridge 

from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors 
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

In [7]:
from sklearn.linear_model import RidgeClassifier


# A Note on Multi-Label classification

Basically, there are three "methods/ways" to solve a multi-label classification problem:

   ### 1) Problem Transformation
   Problem Transformation: transforms our multi-label problem into single-label problem(s). This method can be carried out in three different ways as:
     * Binary Relevance
     * Classifier Chains
     * Label Powerset
   ### 2) Adapted Algorithm
   Adapted Algorithm: adapts the algorithm to directly perform multi-label classification, rather than transforming the problem into different subsets of problems.
     * e.g. kNN's multi-label adaptation is MLkNN. 
     
   ### 3) Ensemble approaches
   Ensemble Approaches: Ensemble apparently always produces better results. 
     * Scikit-Multilearn library provides different ensembling classification functions. 



# Model 1:  Ridge Classifier 

## Problem Transformation- BinaryRelevance

 * Ridge Classifier doesn't support multi-label classification
 * Therefore, you have to implement one of the 3 multi label classification methods
 * We chose to use BinaryRelevance which is a type of "Problem Transformation"

In [ ]:
#import data
X_train = pd.read_pickle('veclyrics_doc2vec_train.pkl')
X_test = pd.read_pickle('veclyrics_doc2vec_test.pkl')
y_train_all = pd.read_pickle('response_doc2vec_train.pkl')
y_test_all = pd.read_pickle('response_doc2vec_test.pkl')
y_train_all = y_train_all.iloc[:,3:10]
y_test_all = y_test_all.iloc[:,3:10]
y_train_one = y_train_all.iloc[:,0]
y_test_one = y_test_all.iloc[:,0]

In [248]:
y_train_ridge = np.asarray(y_train_all)
X_train_ridge = np.asarray(X_train)
X_test_ridge = np.asarray(X_test)
y_test_ridge = np.asarray(y_test_all)

In [238]:
from skmultilearn.problem_transform import BinaryRelevance
import time

start=time.time()
classifier = BinaryRelevance(
    classifier = RidgeClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train_ridge, y_train_all_ridge)

print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  5.0 seconds


In [240]:
y_pred_ridge = classifier.predict(X_test_ridge)

In [241]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_all, y_pred_ridge))
   
    
print("Classification Report for Ridge Classification")

              precision    recall  f1-score   support

           0       0.77      0.99      0.87      7766
           1       0.18      0.00      0.00      1823
           2       0.53      0.33      0.41      4565
           3       0.86      0.00      0.01      1528
           4       0.36      0.01      0.02      2120
           5       0.23      0.00      0.01      1872
           6       0.88      0.08      0.14       743

   micro avg       0.71      0.46      0.56     20417
   macro avg       0.54      0.20      0.21     20417
weighted avg       0.58      0.46      0.43     20417
 samples avg       0.74      0.51      0.56     20417

Classification Report for Ridge Classification


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.



In [242]:
#proportion of 1s in each genre column
y_test_all.sum()/len(y_test_all)

rock                 0.759511
singer-songwriter    0.178289
pop                  0.446455
metal                0.149438
folk                 0.207335
country              0.183081
hip hop / rap        0.072665
dtype: float64

In [243]:
#y_pred_ridge vs y_test

acc = []

for i in range(0,7):
    check = y_pred_ridge[:,i] == np.array(y_test_all.iloc[:,i])
    accuracy = check.sum()/len(check)
    acc.append(accuracy)
    
comp = pd.DataFrame(y_test_all.sum()/len(y_test_all)).rename(columns = {0:'null'})
comp['ridge'] = acc
comp.iloc[1:,0] = 1-comp.iloc[1:,0]
comp['ridge'] = comp['ridge'].divide(10000)
comp['difference'] = (comp['ridge']- comp['null'])


print(comp)

                       null     ridge  difference
rock               0.759511  0.763469    0.003958
singer-songwriter  0.821711  0.839492    0.017781
pop                0.553545  0.535308   -0.018237
metal              0.850562  0.869209    0.018647
folk               0.792665  0.806578    0.013913
country            0.816919  0.832828    0.015909
hip hop / rap      0.927335  0.942559    0.015224


In [260]:
# dir(RidgeClassifier)

In [259]:
#dir(BinaryRelevance)

In [275]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
import time

start=time.time()
classifier = BinaryRelevance(
    classifier = RidgeClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train_ridge, y_train_all_ridge)

print('training time taken: ',round(time.time()-start,0),'seconds')

parameters = [
    {
        'classifier': [RidgeClassifier()],
        'classifier__alpha': [0.001, 0.0001],
    },
]

clf = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy')
clf.fit(X_train_ridge, y_train_ridge)

print (clf.best_params_, clf.best_score_)


training time taken:  5.0 seconds


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.



{'classifier': RidgeClassifier(alpha=0.001, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001), 'classifier__alpha': 0.001} 0.2358550540368722


In [276]:
from skmultilearn.problem_transform import BinaryRelevance
import time

start=time.time()
classifier = BinaryRelevance(
    classifier = RidgeClassifier(alpha=0.001, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)
)
    
classifier.fit(X_train_ridge, y_train_all_ridge)

print('training time taken: ',round(time.time()-start,0),'seconds')


training time taken:  0.0 seconds


In [277]:
y_pred_ridge2 = classifier.predict(X_test_ridge)

In [278]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_all, y_pred_ridge2))
   
    
print("Classification Report for Ridge Classification")

              precision    recall  f1-score   support

           0       0.77      0.99      0.87      7766
           1       0.18      0.00      0.00      1823
           2       0.53      0.33      0.41      4565
           3       0.86      0.00      0.01      1528
           4       0.35      0.01      0.02      2120
           5       0.23      0.01      0.01      1872
           6       0.88      0.08      0.14       743

   micro avg       0.71      0.45      0.56     20417
   macro avg       0.54      0.20      0.21     20417
weighted avg       0.58      0.45      0.43     20417
 samples avg       0.74      0.51      0.56     20417

Classification Report for Ridge Classification


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.



In [279]:
#y_pred_ridge vs y_test

acc = []

for i in range(0,7):
    check = y_pred_ridge2[:,i] == np.array(y_test_all.iloc[:,i])
    accuracy = check.sum()/len(check)
    acc.append(accuracy)
    
comp = pd.DataFrame(y_test_all.sum()/len(y_test_all)).rename(columns = {0:'null'})
comp['ridge'] = acc
comp.iloc[1:,0] = 1-comp.iloc[1:,0]
comp['ridge'] = comp['ridge'].divide(10000)
comp['difference'] = (comp['ridge']- comp['null'])


print(comp)

                       null     ridge  difference
rock               0.759511  0.763676    0.004165
singer-songwriter  0.821711  0.839492    0.017781
pop                0.553545  0.535576   -0.017970
metal              0.850562  0.869209    0.018647
folk               0.792665  0.806520    0.013855
country            0.816919  0.832574    0.015655
hip hop / rap      0.927335  0.942559    0.015224


# Notes on K Nearest Neighbors

 * reference: 
 https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/
 https://www.python-course.eu/k_nearest_neighbor_classifier.php

KNN neighbors is a package of the sklearn, which provides functionalities for nearest neighbor classifiers both for unsupervised and supervised learning.

((it is parametric bc u need to give it at least a K value))

The classes in sklearn.neighbors can handle both Numpy arrays and scipy.sparse matrices as input. For dense matrices, a large number of possible distance metrics are supported. For sparse matrices, arbitrary Minkowski metrics are supported for searches.

scikit-learn implements two different nearest neighbors classifiers:

### KNeighborsClassifier

is based on the k nearest neighbors of a sample, which has to be classified. The number 'k' is an integer value specified by the user. This is the most frequently used classifiers of both algorithms.

### RadiusNeighborsClassifier

is based on the number of neighbors within a fixed radius r for each sample which has to be classified. 'r' is float value specified by the user. This classifier is less often used.
There is no general way to define an optimal value for 'k'. This value depends on the data. As a general rule we can say that increasing 'k' reduces the noise but on the other hand makes the boundaries less distinct.

The decision based on the nearest neighbors can be reached either uniform weights, the class assigned to a query sample is calculated by a simple majority vote of the k-nearest neighbors. This does not take into account that the neighbors closer to the sample should contribute more than the ones further away. The weighting can be controlled by the weights keyword:

weights = 'uniform' assigns uniform weights to each neighbor. This is also the default value.

weights = 'distance' assigns weights proportional to the inverse of the distance from the query sample.

It is also possible to supply a user-defined function to compute the distance.


#### Scaling the data

 - MinMaxScaler: One of the major reason is that your features such as price can't have negative values and as you mentioned, it could be sparse. From Documentation: The motivation to use this scaling include robustness to very small standard deviations of features and preserving zero entries in sparse data.

 - if your numerical variable has a huge variance, then go for RobustScaler or StandardScaler.

 - You dont have to scale the one hot encoded features.

 - For BoW, it is important to preserve the sparsity of the data. If you apply the StandardScaler, you will lose the sparsity. You definitely have to go for MinMaxScaler. Another option would be to go for TfidfVectorizer, which does the l2 normalization by default.
 

#### Comparing Error Rate with the K Value

* There is no way to know beforehand which value of K that yields the best results in the first go. So randomly choose 5 as the K value.

* One way to find the best value of K is to plot the graph of K value and the corresponding error rate for the dataset.

* Below, is a plot of the mean error for the predicted values of test set for all the K values between 1 and 40.

* To do this, we had to first calculate the mean of error for all the predicted values where K ranges from 1 and 40. 



## Parameter Tuning

### KNN Parameters

 - n_neighbors 
   int, optional (default = 5)) 
   Number of neighbors to use by default for meth:'kneighbors' queries.
 
 - weights
   str or callable, optional (default = 'uniform')
   weight function used in prediction. Possible values:
       - 'uniform' : uniform weights. All points in each neighborhood are weighted equally.
       - 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point           will have a greater influence than neighbors which are further away.
       - [callable] : a user-defined function which accepts an array of distances, and returns an array of the same             shape containing the weights.

 - algorithm
    optional Algorithm used to compute the nearest neighbors:
    'ball_tree' will use :class:'BallTree'
    'kd_tree' will use :class:'KDTree'
    'brute' will use a brute-force search.
    'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:'fit' method.
     Note: fitting on sparse input will override the setting of this parameter, using brute force.
 
 - leaf_size
    int, optional (default = 30)
    Leaf size passed to BallTree or KDTree. This can affect the speed of the construction and query, as well as the       memory required to store the tree. The optimal value depends on the nature of the problem.

 - p
    integer, optional (default = 2)
    Power parameter for the Minkowski metric. When p = 1, this is equivalent to using manhattan_distance (l1), and       euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.

 - metric
    string or callable, default 'minkowski'
    the distance metric to use for the tree. The default metric is minkowski, and with p=2 is equivalent to the           standard Euclidean metric. See the documentation of the DistanceMetric class for a list of available metrics.

 - metric_params
   dict, optional (default = None)
   Additional keyword arguments for the metric function.

 - n_jobs
   int, optional (default = 1)
   The number of parallel jobs to run for neighbors search. If '-1', then the number of jobs is set to the number of    CPU cores. Doesn't affect :meth:'fit' method.


# Evaluating KNN

    * For evaluating an algorithm, confusion matrix, precision, recall and f1 score are the most commonly used metrics. The confusion_matrix and classification_report methods of the sklearn.metrics can be used to calculate these metrics. 
    
5.1.1. Generalization quality measures¶
There are several ways to measure a classifier’s generalization quality:

Hamming loss measures how well the classifier predicts each of the labels, averaged over samples, then over labels
accuracy score measures how well the classifier predicts label combinations, averaged over samples

jaccard similarity measures the proportion of predicted labels for a sample to its correct assignment, averaged over samples

precision measures how many samples with ,

recall measures how many samples ,

F1 score measures a weighted average of precision and recall, where both have the same impact on the score
These measures are conveniently provided by sklearn:


#metrics.hamming_loss(y_test_all, predictions_MLkNN) # k = 5

# evaluating in general

* AUC is useful as a single number summary of classifier performance

* Higher value = better classifier

* If you randomly chose one positive and one negative observation, AUC represents the likelihood that your classifier will assign a higher predicted probability to the positive observation

* AUC is useful even when there is high class imbalance (unlike classification accuracy)

# Adapted Algorithm- MLkNN


Multi-label k-nearest neighbour (MLkNN ) [25] is one of the most widely cited
algorithm adaptation approaches. MLkNN is essentially a binary relevance algorithm, which acts on the labels individually, but instead of applying the standard k-nearest neighbour algorithm directly, it combines it with the maximum a posteriori principle. 

Dependent MLkNN (DMLkNN ) [22] follows the same principle
as MLkNN but incorporates all of the labels while deciding the probability for
each label, therefore taking label associations into account.

IBLR-ML [4] is another modification of the k-nearest neighbour algorithm. It finds the nearest
neighbours of the data point to be labeled, and trains a logistic regression model
for each label using the labels of these neighbourhood points as features, thus
taking the label associations into account. 

An algorithmic performance improvement of binary relevance combined with standard k-nearest neighbour, BRkNN,
has also been proposed [16].

In [3]:
#proportion of 1s in each genre column
y_test_all.sum()/len(y_test_all)

rock                 0.759511
singer-songwriter    0.178289
pop                  0.446455
metal                0.149438
folk                 0.207335
country              0.183081
hip hop / rap        0.072665
dtype: float64

In [25]:
y_train_all_mlknn = y_train_all.rename_axis('ID').values #turning into numpy array
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)



print(y_train_all_mlknn)

[[1 1 0 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 1 ... 0 1 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [1 0 1 ... 1 0 0]]


In [ ]:
# have to give MLkNN a numpy array so 

# Multi-learn KNN -  Adapted Algorithm

* ran grid search from k = 10 - 50 
* returned optimal parameters:
    {'k': 10, 's': 0.5} 0.39829904622016976


In [ ]:

#import data
X_train = pd.read_pickle('veclyrics_doc2vec_train.pkl')
X_test = pd.read_pickle('veclyrics_doc2vec_test.pkl')
y_train_all = pd.read_pickle('response_doc2vec_train.pkl')
y_test_all = pd.read_pickle('response_doc2vec_test.pkl')
y_train_all = y_train_all.iloc[:,3:10]
y_test_all = y_test_all.iloc[:,3:10]
y_train_one = y_train_all.iloc[:,0]
y_test_one = y_test_all.iloc[:,0]

In [79]:
y_train_MLkNN50 = y_train_all
y_test_MLkNN = y_test_all


In [81]:
X_train = np.asarray(X_train)
#y_train_MLkNN50 = y_train_MLkNN50.rename_axis('ID').values #turning df into numpy array
y_train_MLkNN50 = np.asarray(y_train_MLkNN50)


In [82]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

parameters = {'k': range(10,50,10), 's': [0.5, 0.7, 1.0]}

clf = GridSearchCV(MLkNN(), parameters, scoring='f1_macro', verbose = 5)
clf.fit(X_train, y_train_MLkNN50)

print (clf.best_params_, clf.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] k=10, s=0.5 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......................... k=10, s=0.5, score=0.406, total= 2.6min
[CV] k=10, s=0.5 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV] ......................... k=10, s=0.5, score=0.393, total= 2.7min
[CV] k=10, s=0.5 .....................................................


...

[CV] ......................... k=10, s=0.7, score=0.406, total= 2.6min
[CV] k=10, s=0.7 .....................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.6min remaining:    0.0s


[CV] ......................... k=10, s=0.7, score=0.393, total= 2.7min
[CV] k=10, s=0.7 .....................................................
[CV] ......................... k=10, s=0.7, score=0.396, total= 2.9min
[CV] k=10, s=1.0 .....................................................
[CV] ......................... k=10, s=1.0, score=0.406, total= 2.7min
[CV] k=10, s=1.0 .....................................................
[CV] ......................... k=10, s=1.0, score=0.393, total= 2.5min
[CV] k=10, s=1.0 .....................................................
[CV] ......................... k=10, s=1.0, score=0.396, total= 2.5min
[CV] k=20, s=0.5 .....................................................
[CV] ......................... k=20, s=0.5, score=0.385, total= 2.8min
[CV] k=20, s=0.5 .....................................................
[CV] ......................... k=20, s=0.5, score=0.385, total= 2.8min
[CV] k=20, s=0.5 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 98.1min finished


{'k': 10, 's': 0.5} 0.39829904622016976


In [ ]:
# took 98.1 min
# 36 fits
# best from 10 to 40 was k = 10, s = 0.5
# 'k': 10, 's': 0.5} 0.39829904622016976

 # MLkNN for s - smoothing parameter
 Resource:
 https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/pr07.pdf
 
 Furthermore, the input argument s is a smoothing parameter controlling the strength of uniform prior (In this paper, s is set to be 1 which yields the Laplace smoothing).
 
 Laplace Smoothing. In statistics, Laplace Smoothing is a technique to smooth categorical data. Laplace Smoothing is introduced to solve the problem of zero probability. from 0 to 1
 
 
 
 # for K
 http://lup.lub.lu.se/luur/download?func=downloadFile&recordOId=8933695&fileOId=8933698
 
 The multi-label methods used in this work were explained in section 2.3 and
were implemented by making use of code created by Min-Ling Zhang [27].
For the MLKNN method the number of neighbours used was chosen to be
30 as was done for the single-label KNN method. The other parameters
for the MLKNN method were chosen to be the default values presented in
[30]. For the ML-RBF method the default values were used as parameters
since changes to these values did not produce any noticeable increase in
classification accuracy. 


In [126]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

parameters = {'k': range(3,5), 's': [0.5, 1.0]}

clf = GridSearchCV(MLkNN(), parameters, scoring='f1_macro', verbose = 5)
clf.fit(X_train, y_train_MLkNN50)

print (clf.best_params_, clf.best_score_)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] k=3, s=0.5 ......................................................
[CV] .......................... k=3, s=0.5, score=0.414, total= 2.6min
[CV] k=3, s=0.5 ......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV] .......................... k=3, s=0.5, score=0.414, total= 2.5min
[CV] k=3, s=0.5 ......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min remaining:    0.0s


...

[CV] .......................... k=3, s=1.0, score=0.414, total= 2.4min
[CV] k=3, s=1.0 ......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.1min remaining:    0.0s


[CV] .......................... k=3, s=1.0, score=0.414, total= 2.4min
[CV] k=3, s=1.0 ......................................................
[CV] .......................... k=3, s=1.0, score=0.411, total= 2.4min
[CV] k=4, s=0.5 ......................................................
[CV] .......................... k=4, s=0.5, score=0.429, total= 2.4min
[CV] k=4, s=0.5 ......................................................
[CV] .......................... k=4, s=0.5, score=0.404, total= 2.5min
[CV] k=4, s=0.5 ......................................................
[CV] .......................... k=4, s=0.5, score=0.452, total= 2.5min
[CV] k=4, s=1.0 ......................................................
[CV] .......................... k=4, s=1.0, score=0.429, total= 2.4min
[CV] k=4, s=1.0 ......................................................
[CV] .......................... k=4, s=1.0, score=0.404, total= 2.5min
[CV] k=4, s=1.0 ......................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 29.9min finished


{'k': 4, 's': 0.5} 0.4280822442015816


In [127]:
# optimal parmaeters {'k': 4, 's': 0.5} 0.4280822442015816


In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

parameters = {'k': range(3,5), 's': [0.5, 1.0]}

clf = GridSearchCV(MLkNN(), parameters, scoring='f1_macro', verbose = 5)
clf.fit(X_train, y_train_MLkNN50)

print (clf.best_params_, clf.best_score_)



predictions_MNB_BR = clf_MNB_BR.predict(X_test_MNB_BR)


In [203]:
import datetime as dt
from sklearn.metrics import precision_score

from sklearn.metrics import accuracy_score, precision_score, f1_score


In [ ]:
y_train_mlknn = y_train_all
y_test_mlknn = y_test_all
X_train_mlknn = np.asarray(X_train)
y_train_mlknn = np.asarray(y_train_mlknn)
X_test_mlknn = np.asarray(X_test)

In [208]:
mlknn = MLkNN(k=4, s=0.5)

begin = dt.datetime.now()

mlknn.fit(X_train_mlknn, y_train_mlknn)
y_pred_mlknn = mlknn.predict(X_test_mlknn)
y_pred_train_mlknn = mlknn.predict(X_train_mlknn)


In [212]:
print(
    "Run time: {};".format(dt.datetime.now() - begin),
    "training accuracy: {};".format(accuracy_score(y_train_mlknn, y_pred_train_mlknn)),
    "training precision: {};".format(precision_score(y_train_mlknn, y_pred_train_mlknn, average = 'weighted')),
    "training F1: {}.".format(f1_score(y_train_mlknn, y_pred_train_mlknn, average = 'weighted')),
    "testing accuracy: {};".format(accuracy_score(y_test_mlknn, y_pred_mlknn)),
    "testing precision: {};".format(precision_score(y_test_mlknn, y_pred_mlknn, average = 'weighted')),
    "testing F1: {}.".format(f1_score(y_test_mlknn, y_pred_mlknn, average = 'weighted'))
)


Run time: 0:09:49.191633; training accuracy: 0.3832216734314636; training precision: 0.7519839057126109; training F1: 0.7232767293857527. testing accuracy: 0.12948655256723715; testing precision: 0.4907048179906204; testing F1: 0.48961869820420817.


In [220]:
from sklearn.metrics import zero_one_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score

def eval_multi_metrics(ytrue, ypred):
    print ("The Accuracy Score is:",accuracy_score(ytrue, ypred))
    print ("The Zero One Loss is:",zero_one_loss(ytrue, ypred))
    print ("The Hamming Loss is:",hamming_loss(ytrue, ypred))

In [ ]:
     """
     INPUT:
    :param ytrue: ground truth
    :param ypred: predictions

     """

In [221]:
eval_multi_metrics(y_test_mlknn, y_pred_mlknn)

The Accuracy Score is: 0.12948655256723715
The Zero One Loss is: 0.8705134474327628
The Hamming Loss is: 0.27334963325183376


In [ ]:
eval_bi_metrics

# Multinomial Naive Bayes - Binary Relevance

* USED MIN MAX SCALER TO REMOVE NEGATIVE VALUES

In [97]:
y_train_MNB_BR = y_train_all
y_test_MNB_BR = y_test_all
X_train_MNB_BR = X_train

In [98]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train_MNB_BR)

X_train_MNB_BR = scaler.transform(X_train_MNB_BR)

X_test_MNB_BR = scaler.transform(X_test)

In [114]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
clf_MNB_BR = BinaryRelevance(MultinomialNB())

# train
clf_MNB_BR.fit(X_train_MNB_BR, y_train_MNB_BR)

# predict
predictions_MNB_BR = clf_MNB_BR.predict(X_test_MNB_BR)

# accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test_MNB_BR,predictions_MNB_BR) 

0.1852322738386308

In [ ]:
print(y_test_MNB_BR.value_counts())

# calculate null accuracy (for multi-class classification problems)
# .head(1) assesses the value 1208
null_accuracy = y_test_MNB_BR.value_counts().head(1) / len(y_test_MNB_BR)
print('Null accuracy:', null_accuracy)

# Manual calculation of null accuracy by always predicting the majority class
print('Manual null accuracy:',(5617 / (5617 + 4609)))

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test_MNB_BR, y_pred_prob)


# Multinomial Naive Bayes - Classifier Chains

 * used MinMaxScaler

In [102]:
# NOT RUN YET

y_train_MNB_cc = y_train_all
y_test_MNB_cc = y_test_all
X_train_MNB_cc = X_train

In [103]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train_MNB_cc)

X_train_MNB_cc = scaler.transform(X_train_MNB_cc)
X_test_MNB_cc = scaler.transform(X_test)

In [115]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import MultinomialNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
clf_MNB_cc = LabelPowerset(MultinomialNB())

# train
clf_MNB_cc.fit(X_train_MNB_cc, y_train_MNB_cc)

# predict
predictions_MNB_cc = clf_MNB_cc.predict(X_test_MNB_cc)

accuracy_score(y_test_MNB_cc,predictions_MNB_cc)

0.1852322738386308

# Multinomial Naive Bayes - Label Powerset

 * Used MinMaxScaler

In [108]:

y_train_MNB_LP = y_train_all
y_test_MNB_LP = y_test_all
X_train_MNB_LP = X_train

In [109]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train_MNB_LP)

X_train_MNB_LP = scaler.transform(X_train_MNB_LP)
X_test_MNB_LP = scaler.transform(X_test)

In [119]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import MultinomialNB

# NOT RUN YET

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
clf_MNB_LP = LabelPowerset(MultinomialNB())

# train
clf_MNB_LP.fit(X_train_MNB_LP, y_train_MNB_LP)

# predict
predictions_MNB_LP = clf_MNB_LP.predict(X_test_MNB_LP)

accuracy_score(y_test_MNB_LP,predictions_MNB_LP)

0.1852322738386308

# Bernoulli Naive Bayes

In [192]:
# MAYBE NOT TO RUN 
'''
gnb = BernoulliNB()
gnb.fit(X_train,y_train_all)
gnb_y_pred = gnb.predict(X_test)
accuracyBNB = gnb.score(X_test,y_test_all)
print("Accuracy for bernoulli naive bayes:", accuracyBNB)
# print the confusion matrix
metrics.confusion_matrix(y_test_all, gnb_y_pred)'''

'\ngnb = BernoulliNB()\ngnb.fit(X_train,y_train_all)\ngnb_y_pred = gnb.predict(X_test)\naccuracyBNB = gnb.score(X_test,y_test_all)\nprint("Accuracy for bernoulli naive bayes:", accuracyBNB)\n# print the confusion matrix\nmetrics.confusion_matrix(y_test_all, gnb_y_pred)'

In [ ]:
# Appendix

# notes for me



#### The below applies to multi-classification problems which is NOT the same as multi-label.

#### OneVsRest - problem transformation method

#### treats the problem like multiple sets of binary classification problems

* Traditional two-class and multi-class problems can both be cast into multi-label ones by restricting each instance to have only one label. On the other hand, the generality of multi-label problems inevitably makes it more difficult to learn. An intuitive approach to solving multi-label problem is to decompose it into multiple independent binary classification problems (one per category).

* In an “one-to-rest” strategy, one could build multiple independent classifiers and, for an unseen instance, choose the class for which the confidence is maximized.

* The main assumption here is that the labels are mutually exclusive. You do not consider any underlying correlation between the classes in this method.

* For instance, it is more like asking simple questions, say, “is the comment toxic or not”, “is the comment threatening or not?”, etc. Also there might be an extensive case of overfitting here, since most of the comments are unlabeled, i,e., most of the comments are clean comments.

# more notes for me
Choosing ML algoriths
We chose machine learning multicategory classification algorithms from the following seven algorithmic families: support vector machines, ridge classification, regularized logistic regression, Bayesian logistic regression, random forests, k-nearest neighbors. 


Since all the classification tasks were multicategory (that is, with three or more classes) and most of the employed classifiers (except for random forests, k-nearest neighbors) are designed for binary classification problems (that is, with two classes), we can adopt a one-versus-rest approach for the latter methods.

We can train separate binary classifiers for each class against the rest and then classify new samples by taking a vote of the binary classifiers and choosing the class with the ‘strongest’ vote. The one-versus-rest approach for classification is known to be among the best performing methods for multicategory classification for other types of data.

Random forests, k-nearest neighbors can solve multicategory problems natively and can be applied directly.